<a href="https://colab.research.google.com/github/pavan-elisetty/bounce_assgn/blob/main/bounce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install haversine 
import pandas as pd
import numpy as np
from haversine import haversine as hav

In [10]:
drop = pd.read_excel('https://github.com/pavan-elisetty/bounce_assgn/blob/main/drop.xlsx?raw=true')
pickup = pd.read_excel('https://github.com/pavan-elisetty/bounce_assgn/blob/main/pickup.xlsx?raw=true')

In [11]:
drop['count']=0
drop['count reach']=False
def count_check_max(x):
  if x>120:
    maxi=8
  elif x>=100 and x<120:
    maxi=4
  elif x>=50 and x<100:
    maxi=3
  elif x>=30 and x<50:
    maxi=2
  elif x>=20 and x<30:
    maxi=1
  elif x>=10 and x<20:
    maxi=1
  else:
    maxi=0
  return maxi


def count_check_min(x):
  if x>120:
    mini=6
  elif x>=100 and x<120:
    mini=2
  elif x>=50 and x<100:
    mini=2
  elif x>=30 and x<50:
    mini=1
  elif x>=20 and x<30:
    mini=1
  elif x>=10 and x<20:
    mini=0
  else:
    mini=0
  return mini
drop2=drop.copy()
drop2['dist']=0
drop2['maxi']=[count_check_max(i) for i in drop2['avg_daily_sessions/supply']]
drop2['mini']=0
drop2['mini']=[count_check_min(i) for i in drop2['avg_daily_sessions/supply']]
drop2['matched']=[[] for i in range(drop2['drop hex'].nunique())]
pickup2=pickup.copy()
pickup2['matched']=False

In [ ]:
for i in range(drop2['drop hex'].nunique()):
  hex=drop2.iloc[i][1]
  lat=drop2.iloc[i][2]
  lon=drop2.iloc[i][3]
  maxi=drop2.iloc[i][8]
  mini=drop2.iloc[i][9]
  if mini==0 or maxi==0:
    continue
  pickup2['dist']=0
  pickup2['dist']=[(hav((lat,lon),(pickup2.iloc[j][2],pickup2.iloc[j][3]),unit='km')) if (pickup2['matched'][j]!=True) else print('') for j in range(len(pickup2))]
  #print(pickup2.head())
  junk=pickup2.loc[pickup2.matched==False]
  tmp=list(junk.nsmallest(mini,'dist')['bike_id'].values)
  #print(tmp)
  
  for j in tmp:
    pickup2.loc[pickup2['bike_id']==j,'matched']=True
    drop2['matched'][i].append(int(j))

In [15]:
drop2['min_reach']=[True if len(drop2['matched'][i])>0 else False for i in range(len(drop2))]

In [16]:
drop2['count']=[drop2['maxi'][i]-drop2['mini'][i] for i in range(len(drop2))]
drop2.head()

,Priority,drop hex,drop hex lat,drop hex lon,avg_daily_sessions/supply,count,count reach,dist,maxi,mini,matched,min_reach
0,1,8960144b4dbffff,13.084574,77.485690,122.071429,2,False,0,8,6,"[53163, 51917, 48693, 50247, 52894, 52876]",True
1,2,8960144b4c3ffff,13.083513,77.482666,112.857143,2,False,0,4,2,"[52480, 50419]",True
2,3,89601696927ffff,13.089918,77.483974,110.500000,2,False,0,4,2,"[53046, 48823]",True
3,4,8960145b547ffff,12.977496,77.570524,90.571429,1,False,0,3,2,"[49529, 41168]",True
4,5,896016965cbffff,13.078991,77.580048,85.571429,1,False,0,3,2,"[46731, 50050]",True


In [ ]:
for i in range(drop2['drop hex'].nunique()):
  hex=drop2.iloc[i][1]
  lat=drop2.iloc[i][2]
  lon=drop2.iloc[i][3]
  maxi=drop2.iloc[i][8]
  mini=drop2.iloc[i][9]
  count=drop2.iloc[i][5]
  if count==0:
    continue
  pickup2['dist']=0
  pickup2['dist']=[(hav((lat,lon),(pickup2.iloc[j][2],pickup2.iloc[j][3]),unit='km')) if (pickup2['matched'][j]!=True) else print('') for j in range(len(pickup2))]
  #print(pickup2.head())
  junk=pickup2.loc[pickup2.matched==False]
  tmp=list(junk.nsmallest(count,'dist')['bike_id'].values)
  min=junk.loc[junk.bike_id==tmp[0]]
  if (hav((lat,lon),(min.lat,min.lon),unit='km'))>4 :
    #print('all more than 4')
    continue
  
  for j in tmp:
    min=junk.loc[junk.bike_id==j]
    if (hav((lat,lon),(min.lat,min.lon),unit='km'))>4 :
      #print('more than 4')
      continue
    pickup2.loc[pickup2['bike_id']==j,'matched']=True
    drop2['matched'][i].append(int(j))
    drop2.loc[drop2['drop hex']==hex,'count']=drop2['count'][i]-1
  

In [18]:
drop2

,Priority,drop hex,drop hex lat,drop hex lon,avg_daily_sessions/supply,count,count reach,dist,maxi,mini,matched,min_reach
0,1,8960144b4dbffff,13.084574,77.485690,122.071429,2,False,0,8,6,"[53163, 51917, 48693, 50247, 52894, 52876]",True
1,2,8960144b4c3ffff,13.083513,77.482666,112.857143,2,False,0,4,2,"[52480, 50419]",True
2,3,89601696927ffff,13.089918,77.483974,110.500000,2,False,0,4,2,"[53046, 48823]",True
3,4,8960145b547ffff,12.977496,77.570524,90.571429,0,False,0,3,2,"[49529, 41168, 50528]",True
4,5,896016965cbffff,13.078991,77.580048,85.571429,0,False,0,3,2,"[46731, 50050, 52919]",True
...,...,...,...,...,...,...,...,...,...,...,...,...
1025,1026,89618924b63ffff,12.911300,77.551372,8.071429,0,False,0,0,0,[],False
1026,1027,8960145909bffff,13.039532,77.552308,8.071429,0,False,0,0,0,[],False
1027,1028,896016960d7ffff,13.095012,77.583327,8.071429,0,False,0,0,0,[],False
1028,1029,8960145843bffff,12.979775,77.526050,8.071429,0,False,0,0,0,[],False
